In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [2]:
# Step 2: Define a dictionary to map class indices to disease names
class_labels = {
    0: 'Acne and Rosacea Photos',
    1: 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
    2: 'Atopic Dermatitis Photos',
    3: 'Bullous Disease Photos',
    4: 'Cellulitis Impetigo and other Bacterial Infections',
    5: 'Eczema Photos',
    6: 'Exanthems and Drug Eruptions',
    7: 'Hair Loss Photos Alopecia and other Hair Diseases',
    8: 'Herpes HPV and other STDs Photos',
    9: 'Light Diseases and Disorders of Pigmentation',
    10: 'Lupus and other Connective Tissue diseases',
    11: 'Melanoma Skin Cancer Nevi and Moles',
    12: 'Nail Fungus and other Nail Disease',
    13: 'Poison Ivy Photos and other Contact Dermatitis',
    14: 'Psoriasis pictures Lichen Planus and related diseases',
    15: 'Scabies Lyme Disease and other Infestations and Bites',
    16: 'Seborrheic Keratoses and other Benign Tumors',
    17: 'Systemic Disease',
    18: 'Tinea Ringworm Candidiasis and other Fungal Infections',
    19: 'Urticaria Hives',
    20: 'Vascular Tumors',
    21: 'Vasculitis Photos',
    22: 'Warts Molluscum and other Viral Infections'
}

In [3]:
# Define constants
IMAGE_SIZE = (224, 224)  # ResNet50 input size
BATCH_SIZE = 128
NUM_CLASSES = len(class_labels)  # The number of classes in your dataset

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [5]:
train_generator = train_datagen.flow_from_directory(
     'D:/skin dataset/train',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 15550 images belonging to 23 classes.


In [6]:

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

test_generator = test_datagen.flow_from_directory(
    'D:/skin dataset/test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 3931 images belonging to 23 classes.


In [7]:
# Create a ResNet50 base model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    'model_checkpoint.h5',  # Specify the file to save the checkpoints
    monitor='val_loss',     # Monitor validation loss
    verbose=1,              # Verbosity level (0 or 1)
    save_best_only=True,    # Save only the best model (based on validation loss)
    mode='min'              # Mode to monitor ('min' for loss, 'max' for accuracy)
)
epochs = 200
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE,
    epochs=epochs
)



In [ ]:
# Evaluate the model on the testing set
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Save the model
model.save('skin_disease_classifier_resnet50.h5')
model.save('skin_disease_classifier_resnet50.keras')
